# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /Users/thomashei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thomashei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomashei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_categories', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # tokenize text
    tokens = word_tokenize(text)
    # remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    # lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# create pipeline with Random Forest Classifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fbed90a75e0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# test model
y_pred = pipeline.predict(X_test)

# print classification report
for i, col in enumerate(Y.columns):
    print(col)
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.71      0.42      0.53      1266
           1       0.83      0.94      0.88      3938
           2       0.39      0.45      0.42        40

    accuracy                           0.81      5244
   macro avg       0.64      0.60      0.61      5244
weighted avg       0.80      0.81      0.79      5244

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.48      0.61       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

The F1-score, a weighted average of precision and recall, reaches its best value at 1 and worst score at 0. There is a huge gap between the result of class 0 and class 1 while class 0 got much higher score than class 0. This is a typical issue of class imbalance. If there are many more examples of one class than the other, it can cause the model to be biased towards predicting the majority class. For example, if there are 90 examples of class 0 and only 10 examples of class 1, a model that always predicts class 0 will still have a 90% accuracy. This can make it seem like the model is performing well, but it may not be able to generalize to new data or perform well on the minority class.

### 6. Improve your model
Use grid search to find better parameters. Few parameters are added in the grid search in order to improve class imbalance and optimize recall.

In [7]:
# set parameters for grid search
param_grid = {
    'tfidf__use_idf': [True, False],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 3],
    'clf__estimator__class_weight': ['balanced', None]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid, verbose=1)

# fit grid search object
cv.fit(X_train, y_train)

# print best parameters
print(cv.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
{'clf__estimator__class_weight': 'balanced', 'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 100, 'tfidf__use_idf': False}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
# test tuned model
y_pred_tuned = cv.predict(X_test)

# print classification report
for i, col in enumerate(Y.columns):
    print(col)
    try:
        print(classification_report(y_test.iloc[:, i], y_pred_tuned[:, i]))
    except IndexError:
        print("Error: y_pred_tuned and y_test have different number of columns")

related
              precision    recall  f1-score   support

           0       0.67      0.50      0.58      1266
           1       0.85      0.92      0.88      3938
           2       0.37      0.53      0.43        40

    accuracy                           0.81      5244
   macro avg       0.63      0.65      0.63      5244
weighted avg       0.81      0.81      0.81      5244

request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      4349
           1       0.77      0.59      0.67       895

    accuracy                           0.90      5244
   macro avg       0.85      0.78      0.80      5244
weighted avg       0.89      0.90      0.89      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      524

/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

After adding parameter to the model and creating grid object to find out the best parameter, some attributes obviously get improvement in class 1, but at the same time the f-1 score of some attributes are even worse than before. However, the f1-socre of class 0 are high in most of the attributes and we won't mind they dropped a bit. The tuned model did improved the issue of class imbalance so we will take this one.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

To try improving the model, we will try another model - SVM classifier. In the first round of code running we got error message. It might be some columns in the target variable that only have one class. To solve the error, we need to found out those columns.

In [9]:
# printing out the unique values for each column in Y
for col in Y.columns:
    print(col, Y[col].unique())

related [1 0 2]
request [0 1]
offer [0 1]
aid_related [0 1]
medical_help [0 1]
medical_products [0 1]
search_and_rescue [0 1]
security [0 1]
military [0 1]
child_alone [0]
water [0 1]
food [0 1]
shelter [0 1]
clothing [0 1]
money [0 1]
missing_people [0 1]
refugees [0 1]
death [0 1]
other_aid [0 1]
infrastructure_related [0 1]
transport [0 1]
buildings [0 1]
electricity [0 1]
tools [0 1]
hospitals [0 1]
shops [0 1]
aid_centers [0 1]
other_infrastructure [0 1]
weather_related [0 1]
floods [0 1]
storm [0 1]
fire [0 1]
earthquake [0 1]
cold [0 1]
other_weather [0 1]
direct_report [0 1]


It seems that the child_alone category has only one unique value of 0. Since this category has no positive examples in the training set, it cannot be used for model training. We are going to drop this column from both the training and testing sets before fitting the model.

In [10]:
# drop 'child_alone' column from Y_train and Y_test
y_train = y_train.drop('child_alone', axis=1)
y_test = y_test.drop('child_alone', axis=1)

In [11]:
# import libraries
from sklearn.svm import LinearSVC

# create new pipeline with SVM classifier
svm_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LinearSVC(random_state=42)))
])

# fit pipeline to training data
svm_pipeline.fit(X_train, y_train)

# test new pipeline
y_pred_svm = svm_pipeline.predict(X_test)

# print classification report for SVM pipeline
for i, col in enumerate(y_train.columns):
    print(col)
    print(classification_report(y_test.iloc[:, i], y_pred_svm[:, i]))

related
              precision    recall  f1-score   support

           0       0.65      0.53      0.58      1266
           1       0.85      0.91      0.88      3938
           2       0.69      0.23      0.34        40

    accuracy                           0.81      5244
   macro avg       0.73      0.55      0.60      5244
weighted avg       0.80      0.81      0.80      5244

request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      4349
           1       0.76      0.58      0.66       895

    accuracy                           0.90      5244
   macro avg       0.84      0.77      0.80      5244
weighted avg       0.89      0.90      0.89      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

Seems the result even worse than the tuned model we've done before. Random Forest should be the better classifier than SVM for this case. Next, we try to add message length as a feature to the original pipeline for improving our model.

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

# define a custom transformer to get message length
class MessageLengthExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [[len(text)] for text in X]

# build a new pipeline with message length as a feature
pipeline2 = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer(use_idf = False))
        ])),
        ('message_length', MessageLengthExtractor())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100,
                                                             min_samples_split = 3,
                                                                class_weight = 'balanced')))
])

In [13]:
# train the pipeline
pipeline2.fit(X_train, y_train)

# test the model
y_pred2 = pipeline2.predict(X_test)

# print classification report
for i, col in enumerate(Y.columns):
    print(col)
    try:
        print(classification_report(y_test.iloc[:, i], y_pred2[:, i]))
    except IndexError:
        print("No samples found for label:", col)

related
              precision    recall  f1-score   support

           0       0.66      0.52      0.58      1266
           1       0.85      0.92      0.88      3938
           2       0.80      0.20      0.32        40

    accuracy                           0.82      5244
   macro avg       0.77      0.55      0.60      5244
weighted avg       0.81      0.82      0.81      5244

request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4349
           1       0.79      0.58      0.67       895

    accuracy                           0.90      5244
   macro avg       0.86      0.78      0.81      5244
weighted avg       0.90      0.90      0.90      5244

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      524

/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/thomashei/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/

Not much differences after added message length as feature. We will stick to the original tuned model.

### 9. Export your model as a pickle file

In [14]:
import pickle

pickle.dump(cv, open('webapp_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.